In [32]:
import xml.etree.cElementTree as ET
import pprint

#audit way tags for unexpected city names in Austin map

osmfile = "sample.osm"

expected = ("Austin", "Pflugerville", "Round Rock")

def is_city_name(elem):
    return (elem.attrib['k'] =="addr:city")

def audit_city(osmfile):
     city_file = open(osmfile, "r")
     for event, elem in ET.iterparse(city_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == 'addr:city':
                    city = tag.attrib['v'].strip()
                    if city not in expected: 
                        print city
                        
                         
     city_file.close()

audit_city("sample.osm")

#audit of sample file shows names which appear to be outside of what would be considered Austin (eg Buda > 60mi away)
#this would require edit specific entries; these can be ignore for analysis purposes in most cases with no material effect

Manchaca
Cedar Park, TX
Buda
Sunset Valley
Buda
Lakeway
Sunset Valley
Cedar Park


In [30]:
"""
Your task in this exercise has two steps:

- audit the OSMFILE and change the variable 'mapping' to reflect the changes needed to fix 
    the unexpected street types to the appropriate ones in the expected list.
    You have to add mappings only for the actual problems you find in this OSMFILE,
    not a generalized solution, since that may and will depend on the particular area you are auditing.
- write the update_name function, to actually fix the street name.
    The function takes a string with street name as an argument and should return the fixed name
    We have provided a simple test so that you see what exactly is expected
"""
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "sample.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Ave.": "Avenue",
            "Rd.": "Road"
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    pprint.pprint (dict(street_types))
    return street_types


def update_name(name, mapping):

    # YOUR CODE HERE
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping.keys():
            print 'Before: ', name
            name = re.sub(m.group(), mapping[m.group()], name)
            print 'After: ', name


    return name


def test():
    st_types = audit(OSMFILE)
    assert len(st_types) == 3
    pprint.pprint(dict(st_types))

    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name
            if name == "West Lexington St.":
                assert better_name == "West Lexington Street"
            if name == "Baldwin Rd.":
                assert better_name == "Baldwin Road"


#if __name__ == '__main__':
    #test()

audit ("sample.osm")

{'101': set(['4207 James Casey st #101']),
 '1327': set(['Farm-to-Market Road 1327']),
 '1431': set(['Farm-to-Market Road 1431', 'Old Farm-to-Market 1431']),
 '150': set(['IH-35 South, #150']),
 '1625': set(['Farm-to-Market Road 1625']),
 '1626': set(['FM 1626', 'Farm-to-Market Road 1626']),
 '1826': set(['Ranch to Market Road 1826']),
 '183': set(['US Highway 183']),
 '2222': set(['Ranch to Market Road 2222']),
 '2769': set(['Farm-to-Market Road 2769']),
 '290': set(['East Highway 290', 'US Highway 290', 'West US Highway 290']),
 '35': set(['North Interstate Highway 35']),
 '620': set(['RM 620', 'Ranch Road 620', 'Ranch-to-Market Road 620']),
 '812': set(['Farm-to-Market Road 812']),
 '969': set(['FM 969']),
 '973': set(['Farm-to-Market Road 973']),
 'A': set(['Avenue A']),
 'Acres': set(['Green Acres']),
 'Affirmed': set(['Affirmed']),
 'Alley': set(['Oak Alley']),
 'Alto': set(['Camino Alto']),
 'Amistad': set(['Circulo de Amistad']),
 'Arbolago': set(['Camino Arbolago']),
 'Ave': s